In [405]:
import numpy as np
import matplotlib.pyplot as plt

In [406]:
class Grid:    # Environment
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
    
    def set(self, rewards, actions):
        # rewards should be a dictionary of: (i, j): r, or: (row, col): reward
        # actions should be a dictionary of: (i, j): A or (row, col): list of possible actions
        self.rewards = rewards
        self.actions = actions
        
    def set_state(self, s):
        # The state s is the location of the player in the grid: s = (i, j)
        self.i = s[0]
        self.j = s[1]
        
    def current_state(self):
        return (self.i, self.j)
    
    def is_terminal(self, s):
        # A terminal state won't be in the actions dictionary (since it won't have any associated action)
        return s not in self.actions
    
    def move(self, action):
        # check if legal move first
        # Possible actions: U/D/L/R
        if action in self.actions[self.current_state()]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        # return reward (if any)
        return self.rewards.get(self.current_state(), 0)
    
    def undo_move(self, action):
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        # raise an exception if we arrive somewhere we shouldn't be
        # should never happen
        assert(self.current_state() in self.all_states())
        
    def game_over(self):
        # The game is over if we are in a state where no action is possible
        return self.current_state() not in self.actions
    
    def all_states(self):
        # Cast to a set to avoid repetition in states
        return set(list(self.rewards.keys()) + list(self.actions.keys()))
    
    def draw_grid(self):
        states = self.all_states()
        for i in range(self.height):
            for j in range(self.width):
                s = (i, j)
                symbol = ''
                if s in states:
                    if self.current_state() == s:
                        symbol = 's'
                    else:
                        symbol = '.'
                else:
                    symbol = 'x'
                print(symbol, end = '')
                if j != self.width - 1:
                    print('   ', end = '')
            print('\n')

In [407]:
def standard_grid():
    # Define a grid that describes the reward for arriving at each state
    #     and possible actions at each state
    # The grid looks like this
    # x means you can't go there
    # s means start position
    # number means reward at that state
    # .   .   .   1
    # .   x   .  -1
    # s   .   .   .
    g = Grid(4, 3, (2, 0))
    rewards = {(0, 3): 1, (1, 3): -1}
    actions = {
        (0,0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('D', 'L', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('U', 'L', 'R'),
        (2, 3): ('U', 'L')
    }
    g.set(rewards, actions)
    return g

In [408]:
SMALL_ENOUGH = 10e-4    # threshold for convergence

In [409]:
# V is the value function dictionary, and g is the grid (environment)
def print_values(V, g):
    for i in range(g.height):
        print('-------------------------')
        print('|', end = "")
        for j in range(g.width):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end = "")
            else:
                print("%.2f|" % v, end = "")    # negative sign takes up an extra space
        print()
    print('-------------------------')

In [410]:
# P is the policy dictionary (Mapping each state to the action to take)
def print_policy(P, g):
    for i in range(g.height):
        print('-------------------------')
        print('|', end = '')
        for j in range(g.width):
            a = P.get((i, j), ' ')
            print('  %s  |' % a, end = '')
        print()
    print('-------------------------')

In [411]:
grid = standard_grid()

In [412]:
states = grid.all_states()

In [413]:
### Iterative Policy Evaluation
### Policy: Uniformely random actions
### In this case, the probability of performing action a in state s is 1 / len(grid.actions[s])
### The action state transitions are still deterministic (if you are in state s and you perform action a, you only end up in one possible state s')

# initialize V(s) = 0
V = {}
for s in states:
    V[s] = 0
gamma = 1.0

while True:
    delta = 0    # max change in value function
    for s in states:
        if not grid.is_terminal(s):
            grid.set_state(s)
            old_v = V[s]
            V[s] = 0
            prob_a_s = 1 / len(grid.actions[s])
            for action in grid.actions[s]:
                # perform action to get the reward and the next state's value, then undo action
                r = grid.move(action)
                v_s_prime = V[grid.current_state()]
                V[s] += r + gamma * v_s_prime
                grid.undo_move(action)
            V[s] *= prob_a_s
            delta = max(delta, abs(V[s] - old_v))
    if delta < SMALL_ENOUGH:
        break

In [421]:
print_values(V, grid)

-------------------------
|-0.03| 0.09| 0.22| 0.00|
-------------------------
|-0.16| 0.00|-0.44| 0.00|
-------------------------
|-0.29|-0.41|-0.54|-0.77|
-------------------------


In [415]:
grid.current_state()

(0, 2)

In [416]:
grid.draw_grid()

.   .   s   .

.   x   .   .

.   .   .   .



In [417]:
policy = {
    (0,0):'R',
    (0, 1): 'R',
    (0, 2):'R',
    (1, 0):'U',
    (1, 2):'R',
    (2, 0):'U',
    (2, 1):'R',
    (2, 2):'R',
    (2, 3):'U'
}

In [418]:
### Iterative Policy Evaluation
### Here the policy is given (deterministic)
### The action state transitions are still deterministic (if you are in state s and you perform action a, you only end up in one possible state s')

# initialize V(s) = 0
V2 = {}
for s in states:
    V2[s] = 0
gamma = 0.9

while True:
    delta = 0
    for s in states:
        if not grid.is_terminal(s):
            grid.set_state(s)
            old_v = V2[s]
            # In this case, there is only one action per state, so no need for a for loop
            action = policy[s]
            r = grid.move(action)
            v_s_prime = V2[grid.current_state()]
            V2[s] = r + gamma * v_s_prime
            grid.undo_move(action)
            
            delta = max(delta, abs(V2[s] - old_v))
    if delta < SMALL_ENOUGH:
        break

In [419]:
print_values(V2, grid)

-------------------------
| 0.81| 0.90| 1.00| 0.00|
-------------------------
| 0.73| 0.00|-1.00| 0.00|
-------------------------
| 0.66|-0.81|-0.90|-1.00|
-------------------------


In [420]:
print_policy(policy, grid)

-------------------------
|  R  |  R  |  R  |     |
-------------------------
|  U  |     |  R  |     |
-------------------------
|  U  |  R  |  R  |  U  |
-------------------------


In [390]:
def negative_grid():
    # Define a grid that describes the reward for arriving at each state
    #     and possible actions at each state
    # The grid looks like this
    # x means you can't go there
    # s means start position
    # number means reward at that state
    # .   .   .   1
    # .   x   .  -1
    # s   .   .   .
    g = Grid(4, 3, (2, 0))
    actions = {
        (0,0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('D', 'L', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('U', 'L', 'R'),
        (2, 3): ('U', 'L')
    }
    rewards = {(0, 3): 1, (1, 3): -1}
    
    # Penalise the player for each step, to see if he can finish with the minimum step
    for s in actions.keys():
        rewards[s] = -0.1
    g.set(rewards, actions)
    return g

In [391]:
g = negative_grid()

In [392]:
print_values(g.rewards, g)

-------------------------
|-0.10|-0.10|-0.10| 1.00|
-------------------------
|-0.10| 0.00|-0.10|-1.00|
-------------------------
|-0.10|-0.10|-0.10|-0.10|
-------------------------


In [393]:
ALL_POSSIBLE_ACTIONS = ['U', 'D', 'L', 'R']

In [395]:
# Get the value function corresponding to the given policy
### Iterative Policy Evaluation
### Here the policy is given (deterministic)
### The action state transitions are still deterministic (if you are in state s and you perform action a, you only end up in one possible state s')
def get_value_function(policy, g):
    states = g.all_states()
    # initialize V(s) = 0
    V3 = {}
    for s in states:
        V3[s] = 0
    gamma = 0.9
    while True:
        delta = 0
        for s in states:
            if not g.is_terminal(s):
                g.set_state(s)
                old_v = V3[s]
                action = policy[s]
                r = g.move(action)
                v_s_prime = V3[g.current_state()]
                V3[s] = r + gamma * v_s_prime
                g.undo_move(action)
                
                delta = max(delta, abs(V3[s] - old_v))
        if delta < SMALL_ENOUGH:
            break
    return V3
        

In [396]:
print_policy(policy, g)

-------------------------
|  R  |  L  |  L  |     |
-------------------------
|  D  |     |  R  |     |
-------------------------
|  R  |  R  |  U  |  L  |
-------------------------


In [397]:
V3 = get_value_function(policy, g)
print_values(V3, g)

-------------------------
|-1.00|-1.00|-1.00| 0.00|
-------------------------
|-1.00| 0.00|-1.00| 0.00|
-------------------------
|-1.00|-1.00|-1.00|-1.00|
-------------------------


In [401]:
### Policy Iteration Algorithm
### Finds the optimal policy, starting with a random policy and ameliorating it
def policy_iteration(g):
    policy = {}
    
    # Randomly initialize policy
    # Since the policy is a state -> action mapping, randomly initializing the policy is randomly choosing an action for each non terminal state
    for s in g.actions.keys():
        policy[s] = np.random.choice(g.actions[s])
        
        
    policy_changed = True
        
    # Keep updating the policy (by finding better actions for each state) until the policy doesn't change anymore
    while policy_changed:
        V3 = get_value_function(policy, g)
        policy_changed = False
        gamma = 0.9
        states = g.all_states()
        for s in states:
            if not g.is_terminal(s):
                g.set_state(s)
                old_a = policy[s]    # Old action determined by the old policy
                # Find the best action (action with highest value)
                max_a = old_a
                max_val = float('-inf')
                for a in g.actions[s]:
                    # Perform action, get value, then undo action
                    r = g.move(a)
                    val = r + gamma * V3[g.current_state()]
                    if val > max_val:
                        max_a = a
                        max_val = val
                    g.undo_move(a)
                policy[s] = max_a
                # if the new action is different from the one deyermined by the old policy, then the policy has changed
                if policy[s] != old_a:
                    policy_changed = True
    return policy, V3

In [402]:
opt_policy, opt_val_func = policy_iteration(g)

In [403]:
print_policy(opt_policy, grid)

-------------------------
|  R  |  R  |  R  |     |
-------------------------
|  U  |     |  U  |     |
-------------------------
|  U  |  R  |  U  |  L  |
-------------------------


In [404]:
print_values(opt_val_func, grid)

-------------------------
| 0.62| 0.80| 1.00| 0.00|
-------------------------
| 0.46| 0.00| 0.80| 0.00|
-------------------------
| 0.31| 0.46| 0.62| 0.46|
-------------------------
